## Data Cleaning with spark
### in this notebook the useless columns will be removed

### **PLEASE NOTE :**  
### Since this script stores the results in hadoop, execute it only once, otherwise an error will be thrown

---

### Import Libraries

In [1]:
# import libraries
import pandas as pd
import pyspark as ps
from pyspark.sql.functions import col, sum
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import findspark

### Initialize Spark

In [2]:
# Locate the spark installation
findspark.init()

# Initialize a SparkContext
spark = SparkSession.builder.appName("data_cleaning").getOrCreate()
spark.stop()
sc = ps.SparkContext(appName="data_cleaning")

# Initialize the Session
spark_session = ps.sql.SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/08 12:00:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Connect and import data from HDFS directly into a Spark DataFrame

In [3]:
# Define schema for better manipulation

data_schema = StructType([
    StructField("Title", StringType(), True),
    StructField("description", StringType(), True),
    StructField("authors", StringType(), True),
    StructField("image", StringType(), True),
    StructField("previewLink", StringType(), True),
    StructField("publisher", StringType(), True),
    StructField("publishedDate", StringType(), True),
    StructField("infoLink", StringType(), True),
    StructField("categories", StringType(), True),
    StructField("ratingsCount", FloatType(), True)
])

ratings_schema = StructType([
    StructField("Id", IntegerType(), True),
    StructField("Title", StringType(), True),
    StructField("Price", FloatType(), True),
    StructField("User_id", IntegerType(), True),
    StructField("profileName", StringType(), True),
    StructField("review/helpfulness", StringType(), True),
    StructField("review/score", FloatType(), True),
    StructField("review/time", IntegerType(), True),
    StructField("review/summary", StringType(), True),
    StructField("review/text", StringType(), True)
])

# Load the original data

df_data = spark_session.read.option('escape', '"').csv(
    'hdfs://localhost:9900/user/book_reviews/original_data/books_data.csv', header=True, schema=data_schema)
df_ratings = spark_session.read.option('escape', '"').csv(
    'hdfs://localhost:9900/user/book_reviews/original_data/books_rating.csv', header=True, schema=ratings_schema)

## Data Transformations
---

### - Remove useless columns

These are the columns whcih are not useful for our analysis. The original files are kept unchanged in HDFS, and the new files are stored in HDFS as well.

**Data Table:**
All the links are removed.
- image
- previewLink
- infoLink
- ratingsCount

**Rating Table:**
- id

In [4]:
# Remove image column from data
df_data = df_data.drop(df_data.image)

# Remove previewLink column from data
df_data = df_data.drop(df_data.previewLink)

# Remove infoLink column from data
df_data = df_data.drop(df_data.infoLink)

# Remove ratingsCount column from data
df_data = df_data.drop(df_data.ratingsCount)

# Show the results
df_data.show(5)

# Remove Id column from ratings data
df_ratings = df_ratings.drop(df_ratings.Id)

# Show the results
df_ratings.show(5)

+--------------------+--------------------+-------------------+---------+-------------+--------------------+
|               Title|         description|            authors|publisher|publishedDate|          categories|
+--------------------+--------------------+-------------------+---------+-------------+--------------------+
|Its Only Art If I...|                null|   ['Julie Strain']|     null|         1996|['Comics & Graphi...|
|Dr. Seuss: Americ...|Philip Nel takes ...|     ['Philip Nel']|A&C Black|   2005-01-01|['Biography & Aut...|
|Wonderful Worship...|This resource inc...|   ['David R. Ray']|     null|         2000|        ['Religion']|
|Whispers of the W...|Julia Thomas find...|['Veronica Haddon']|iUniverse|      2005-02|         ['Fiction']|
|Nation Dance: Rel...|                null|    ['Edward Long']|     null|   2003-03-01|                null|
+--------------------+--------------------+-------------------+---------+-------------+--------------------+
only showing top 5 

### - Remove all the punctuation inside each column

This is to avoid parsing problem when the csv in read

In [5]:
from pyspark.sql.functions import col, sum, regexp_replace
import string
punctuations = string.punctuation

data_cols_to_change = ['Title', 'description',
                       'authors', 'publisher', 'categories']

for col_name in data_cols_to_change:
    df_data = df_data.withColumn(col_name, regexp_replace(
        col(col_name), r'[\t]', ' '))

ratings_cols_to_change = ['Title', 'profileName',
                          'review/summary', 'review/text']
for col_name in ratings_cols_to_change:
    df_ratings = df_ratings.withColumn(col_name, regexp_replace(
        col(col_name), r'[\t]', ' '))

In [6]:
# # Check if a given column contains a given character

# contains_A = df_ratings.filter(col("review/text").contains("\t")).count() > 0
# print("Does the 'name' column contain 'A'? ", contains_A)

Does the 'name' column contain 'A'?  False


### Store the results in hadoop

In [7]:
df_ratings.repartition(1).write.csv(
    'hdfs://localhost:9900/user/book_reviews/books_rating_cleaned', mode='overwrite', header=True, sep='\t')

df_data.repartition(1).write.csv(
    'hdfs://localhost:9900/user/book_reviews/books_data_cleaned', mode='overwrite', header=True, sep='\t')

---

## Read the new data to check soundness

In [8]:
data_df = spark_session.read.csv(
    'hdfs://localhost:9900/user/book_reviews/books_data_cleaned', header=True, inferSchema=True, sep='\t')
rating_df = spark_session.read.csv(
    'hdfs://localhost:9900/user/book_reviews/books_rating_cleaned', header=True, inferSchema=True, sep='\t')

In [9]:
data_df.show(5)
#data_df.printSchema()
print("Num values :",data_df.count())
#data_df.describe().show()

data_df.filter(data_df['Title'].startswith('17 Contemporary Christian')).show()

+--------------------+--------------------+-------------------+---------+-------------+--------------------+
|               Title|         description|            authors|publisher|publishedDate|          categories|
+--------------------+--------------------+-------------------+---------+-------------+--------------------+
|Its Only Art If I...|                null|   ['Julie Strain']|     null|         1996|['Comics & Graphi...|
|Dr. Seuss: Americ...|Philip Nel takes ...|     ['Philip Nel']|A&C Black|   2005-01-01|['Biography & Aut...|
|Wonderful Worship...|This resource inc...|   ['David R. Ray']|     null|         2000|        ['Religion']|
|Whispers of the W...|Julia Thomas find...|['Veronica Haddon']|iUniverse|      2005-02|         ['Fiction']|
|Nation Dance: Rel...|                null|    ['Edward Long']|     null|   2003-03-01|                null|
+--------------------+--------------------+-------------------+---------+-------------+--------------------+
only showing top 5 

In [10]:
rating_df.show(5)
#rating_df.printSchema()
print("Num values :",rating_df.count())
#rating_df.describe().show()

rating_df.filter(rating_df['Title'].startswith('17 Contemporary Christian')).show()

+--------------------+-----+-------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|Price|User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+-----+-------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|Its Only Art If I...| null|   null|Jim of Oz "jim-of...|               7/7|         4.0|  940636800|Nice collection o...|This is only for ...|
|Dr. Seuss: Americ...| null|   null|       Kevin Killian|             10/10|         5.0| 1095724800|   Really Enjoyed It|I don't care much...|
|Dr. Seuss: Americ...| null|   null|        John Granger|             10/11|         5.0| 1078790400|Essential for eve...|If people become ...|
|Dr. Seuss: Americ...| null|   null|Roy E. Perry "ama...|               7/7|         4.0| 1090713600|Phlip Nel gives s...|Theodore Seuss

Num values : 3000000


+--------------------+-----+-------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|Price|User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+-----+-------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|17 Contemporary C...|14.99|   null|Arena Connery "Ar...|               1/1|         5.0| 1144972800|not just easy pia...|Yeah, this book i...|
+--------------------+-----+-------+--------------------+------------------+------------+-----------+--------------------+--------------------+



## Join the Tables with Spark

In [11]:
spark_session.stop()